In [1]:
pip install ale-py gym gym[Atari] tensorflow matplotlib pyglet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import random
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from collections import deque
from itertools import islice

In [3]:
env = gym.make('ALE/SpaceInvaders-v5', render_mode='human')
env.reset()

array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0],
        ...,
        [ 0,  0,  0],
        [ 0,  0,  0],
        [ 0,  0,  0]],

       ...,

       [[80, 89, 22],
        [80, 89, 22],
        [80, 89, 22],
        ...,
        [80, 89, 22],
        [80, 89, 22],
        [80, 89, 22]],

       [[80, 89, 22],
        [80, 89, 22],
        [80, 89, 22],
        ...,
        [80, 89, 22],
        [80, 89, 22],
        [80, 89, 22]],

       [[80, 89, 22],
        [80, 89, 22],
        [80, 89, 22],
        ...,
        [80, 89, 22],
        [80, 89, 22],
        [80, 89, 22]]], dtype=uint8)

In [4]:
state_shape = env.observation_space.shape
action_shape = env.action_space.n

In [5]:
#################################################################################
# Hyperparameters

BATCH_SIZE = 64
REPLAY_SIZE = 2000
EPISODES = 300
TARGET_MODEL_UPDATE = 200
REPLAY_MEMORY = 50_000

# CNN model params
LEARNING_RATE = 0.003
KERNEL_SIZE = [8,4,3]
STRIDES = [4,2,1]
POOL_SIZE = 2

# Q-learning params
Q_LEARNING_RATE = 0.2
DISCOUNT_FACTOR = 0.99

In [6]:
#################################################################################
# Model - Create a convolutional neural network with Keras

def network(state_shape, action_shape):
    
    initializer = keras.initializers.HeUniform()
    model = keras.Sequential()

    # Input layer
    model.add(keras.layers.Conv2D(32, kernel_size=KERNEL_SIZE[0], input_shape=(4,85,80,1), activation='relu', 
        padding='same', strides=STRIDES[0], kernel_initializer=initializer))
    #model.add(keras.layers.MaxPooling2D(pool_size=POOL_SIZE))

    # Hidden convolutional layers
    model.add(keras.layers.Conv2D(64, kernel_size=KERNEL_SIZE[1], activation='relu', padding='same', 
        strides=STRIDES[0], kernel_initializer=initializer))
    #model.add(keras.layers.MaxPooling2D(pool_size=POOL_SIZE))
    model.add(keras.layers.Conv2D(64, kernel_size=KERNEL_SIZE[2], activation='relu', padding='same', 
        strides=STRIDES[0], kernel_initializer=initializer))
    #model.add(keras.layers.MaxPooling2D(pool_size=POOL_SIZE))

    # Flatten and use fully connected network
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu', kernel_initializer=initializer))

    # Output layer
    model.add(keras.layers.Dense(action_shape, activation='softmax', kernel_initializer=initializer))

    model.compile(loss=keras.losses.Huber(), optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])
    
    return model

In [7]:
model = network(state_shape, action_shape)
target_model = network(state_shape, action_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 22, 20, 32)     2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 6, 5, 64)       32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 2, 2, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 18)                9234      
                                                                 
Total params: 605,874
Trainable params: 605,874
Non-trai

In [8]:
#################################################################################
# Preprocessing

def preprocess(obs, normalize=False):
    # Crop out score and floor
    img = obs[25:195]  

    # Downsize
    img = img[::2, ::2]

    # Take greyscale (black and white)
    img = img.mean(axis=2)  

    # color = np.array([210, 164, 74]).mean()
    # img[img==color] = 0  
    # img[img==144] = 0
    # img[img==109] = 0
    img[img != 0] = 1

    # Is this needed? normalize the image from -1 to +1  
    # No difference visually but tensor is different
    if normalize:
        img = (img - 128) / 128 - 1  

    # print("before: ", obs.shape)
    # print("after: ", img.shape)

    # reshape to 1D tensor
    return img.reshape(85,80,1)

# frame stacking
# https://danieltakeshi.github.io/2016/11/25/frame-skipping-and-preprocessing-for-deep-q-networks-on-atari-2600-games/
# https://arxiv.org/pdf/1312.5602.pdf
# need to get overlapping sets of frames
# Ex: X1, X2, ... , X7 -> [X1, X2, X3, X4], [X2, X3, X4, X5], ... , [X4, X5, X6, X7]

frame_skip = 4 # only one every four screenshot is considered. If there is no subsampling, not enough information to discern motion
frame_stack_size = 4

def stack_frames(stacked_frames, state, is_new):
    frame = preprocess(state)
    if is_new: # new episode
        # replace stacked_frames with 4 copies of current frame
        for i in range(frame_stack_size):
            stacked_frames.append(frame)
    else:
        # take elementwise maxima of newest frame in stacked_frames and frame
        stacked_frames.append(np.maximum(stacked_frames[3],frame))
    stacked_state = np.stack(stacked_frames)
    return stacked_state, stacked_frames

In [9]:
#################################################################################
# Training agent

def train(env, replay_memory, model, target_model):

    if len(replay_memory) <= REPLAY_SIZE:
        return

    batch = random.sample(replay_memory, BATCH_SIZE)
    states = np.array([step[0] for step in batch])
    q_values = model.predict(states)
    succesive_states = np.array([step[3] for step in batch])
    succesive_q_values = target_model.predict(succesive_states)

    X_train = []
    Y_train = []

    for i, (state, action, reward, new_state, done) in enumerate(batch):
        if not done:
            # Bellman Equation : r(s) + gamma * max_a'(Q(s',a'))
            qValue = reward + DISCOUNT_FACTOR * np.max(succesive_q_values[i])
        else:
            # Pick reward as the episode has ended; no succesive state
            qValue = reward
        
        # TODO: Figure out y_train values work or not
        # Temporal Difference
        # q_value_arr for a state s : [qVal action1, qval action1, ..., qval action18] 
        q_value_arr = q_values[i]
        # Qvalue for action a  : Q(s,a) + alpha(r(s) + gamma*max_a'(Q(s',a')) - Q(s, a))         
        q_value_arr[action] = q_value_arr[action] + LEARNING_RATE * (qValue - q_value_arr[action])

        X_train.append(state)
        Y_train.append(q_value_arr)
    
    model.fit(np.array(X_train), np.array(Y_train), batch_size=BATCH_SIZE)

In [ ]:
#################################################################################
# Deep Q-Learning agent

epsilon = 1
eps_min = 0.05
eps_max = 1
decay = 0.01

model = network(state_shape, action_shape)
target_model = network(state_shape, action_shape)
target_model.set_weights(model.get_weights())

# initialize with zeroes
stacked_frames = deque(maxlen = frame_stack_size)
for i in range(frame_stack_size):
    stacked_frames.append([np.zeros((85,80), dtype=int)])

# Memory buffer to store the last N experiences
replay_memory = deque(maxlen=REPLAY_MEMORY)

update_target_counter = 0
step_counter = 0

for episode in range(EPISODES):
    state = env.reset()
    score = 0 
    done = False

    stacked_state, stacked_frames = stack_frames(stacked_frames, state, True)

    while not done:
        step_counter += 1

        # Epsilon Greedy Strategy with explore probability epsilon
        if np.random.rand() <= epsilon:
            # Explore 
            action = env.action_space.sample()
        else:
            # Exploit best action from cnn
            predictions = model.predict(np.array([stacked_state,])).flatten()
            action = np.argmax(predictions)


        new_state, reward, done, info = env.step(action)
        new_stacked_state, stacked_frames = stack_frames(stacked_frames, new_state, False)

        if step_counter % 4 == 0 or done:
            replay_memory.append([stacked_state, action, reward, new_stacked_state, done])
            train(env, replay_memory, model, target_model)

        score += reward
        stacked_state = new_stacked_state
        state = new_state

        if update_target_counter >= TARGET_MODEL_UPDATE:
                update_target_counter = 0
                target_model.set_weights(model.get_weights())

        if done:
              print('Score: {} after epsidoe = {}'.format(score, episode))

    # Exponential Decay for epsilon (explore with atleast eps_min probability)
    epsilon = eps_min + (eps_max - eps_min) * np.exp(-decay * episode)

Score: 20.0 after epsidoe = 0
Score: 390.0 after epsidoe = 1
Score: 125.0 after epsidoe = 2
Score: 190.0 after epsidoe = 3
Score: 270.0 after epsidoe = 4
Score: 160.0 after epsidoe = 5
Score: 165.0 after epsidoe = 6
Score: 340.0 after epsidoe = 7
Score: 135.0 after epsidoe = 8
Score: 65.0 after epsidoe = 9
Score: 155.0 after epsidoe = 10
Score: 110.0 after epsidoe = 11
Score: 75.0 after epsidoe = 12
Score: 215.0 after epsidoe = 13
Score: 80.0 after epsidoe = 14
Score: 35.0 after epsidoe = 15
Score: 45.0 after epsidoe = 16
Score: 210.0 after epsidoe = 17
Score: 155.0 after epsidoe = 18
1/1 [==============================] - 0s 26ms/step - loss: 1.5893e-06 - accuracy: 1.0000
Score: 360.0 after epsidoe = 19
1/1 [==============================] - 0s 9ms/step - loss: 4.3249e-06 - accuracy: 1.0000


1/1 [==============================] - 0s 45ms/step - loss: 1.2276e-07 - accuracy: 1.0000


1/1 [==============================] - 0s 45ms/step - loss: 1.3102e-06 - accuracy: 1.0000
Score: 125.0 after epsidoe = 20
1/1 [==============================] - 0s 45ms/step - loss: 3.5760e-06 - accuracy: 1.0000


1/1 [==============================] - 0s 9ms/step - loss: 1.7090e-06 - accuracy: 1.0000
Score: 80.0 after epsidoe = 21
1/1 [==============================] - 0s 46ms/step - loss: 9.1446e-07 - accuracy: 1.0000
Score: 120.0 after epsidoe = 22
1/1 [==============================] - 0s 24ms/step - loss: 1.0888e-08 - accuracy: 1.0000


1/1 [==============================] - 0s 25ms/step - loss: 1.7108e-06 - accuracy: 1.0000
Score: 35.0 after epsidoe = 23
1/1 [==============================] - 0s 25ms/step - loss: 1.2968e-06 - accuracy: 1.0000


...

1/1 [==============================] - 0s 25ms/step - loss: 1.5972e-06 - accuracy: 1.0000
Score: 120.0 after epsidoe = 30
1/1 [==============================] - 0s 45ms/step - loss: 1.3097e-06 - accuracy: 1.0000


1/1 [==============================] - 0s 25ms/step - loss: 1.8067e-06 - accuracy: 1.0000
Score: 20.0 after epsidoe = 31
1/1 [==============================] - 0s 47ms/step - loss: 1.3079e-06 - accuracy: 1.0000


1/1 [==============================] - 0s 44ms/step - loss: 6.0363e-06 - accuracy: 1.0000
Score: 80.0 after epsidoe = 32
1/1 [==============================] - 0s 45ms/step - loss: 4.1850e-07 - accuracy: 1.0000


1/1 [==============================] - 0s 46ms/step - loss: 9.1303e-07 - accuracy: 1.0000
Score: 150.0 after epsidoe = 33
1/1 [==============================] - 0s 19ms/step - loss: 2.2527e-07 - accuracy: 1.0000


1/1 [==============================] - 0s 26ms/step - loss: 2.0156e-06 - accuracy: 1.0000


In [ ]:
env.close()

In [ ]:
model.save("models/model1")

In [ ]:
def test():
    env = gym.make('ALE/SpaceInvaders-v5', render_mode='human')
    state = env.reset()

    TEST_EPISODES = 100

    model = keras.models.load_model("models/model1")
    scores = []
    
    # initialize with zeroes
    stacked_frames = deque(maxlen = frame_stack_size)
    for i in range(frame_stack_size):
        stacked_frames.append([np.zeros((85,80), dtype=int)])

    for episode in range(TEST_EPISODES):
        state = env.reset()
        score = 0 
        done = False

        stacked_state, stacked_frames = stack_frames(stacked_frames, state, True)

        while not done:
            predictions = model.predict(np.array([stacked_state,])).flatten()
            action = np.argmax(predictions)


            new_state, reward, done, info = env.step(action)
            new_stacked_state, stacked_frames = stack_frames(stacked_frames, new_state, False)

            score += reward
            stacked_state = new_stacked_state
            state = new_state

            if done:
                scores.append(score)
                print('episode: {}, score: {}'.format(episode, score))

    
    x = np.array([i for i in range(TEST_EPISODES)])
    y = np.array(scores)

    print(np.average(y))
    
    plt.plot(x, y)
    plt.show()

In [ ]:
test()